> In the book Doubly Linked List
>  
> Here seperate lists of vertexes, faces, half-edges
>  
> "Pointers" simply indexes in appropriate list //best of my ideas yet


### Classes for vertexes, faces, half-edges

In [3]:
class Vertex:
    def __init__(self, coord, incident_edge):
        # tuple with coordinates
        self.coord = coord 
        
        # index of half-edge with this vertex as origin  -- not exactly sure how this should work
        self.incident_edge = incident_edge
        
        # index of half_edge immediately to the left of the vertex computed during sweep
        self.edge_to_the_left = None
    
    def set_edge_to_the_left(self, edge_index):
        self.edge_to_the_left = edge_index

        
        
class Face:
    def __init__(self, index, outer_component, inner_components):
        self.face_index = index
        
        # index of and half-edge on outer boundary
        self.outer = outer_component
        
        # list of indexes of one (outer) half-edge in every hole inside this face
        self.inner = inner_components
        
        
class Half_edge:
    def __init__(self, origin, twin, nxt, prv):
        # vertex this half_edge starts from
        self.origin = origin
        
        # opposite half_edge (from destination(this) to origin(this))
        self.twin = twin
        
        # face laying to the left of half_edge
        self.incident_face = None
        
        # next half_edge that is also a boundary to self.incident_face
        self.next = nxt
        
        # previous half_edge that is also a boundary to self.incident_face
        self.prev = prv
        
        # refferal needed in face computing
        self.boundary_index = None
    
    def set_face(self, f):
        self.incident_face = f
        
    def set_boundary(self, boundary_index):
        self.boundary_index = boundary_index
        
    # self.destination = self.twin.origin hence needs not to be used
        

### Subdivision



In [4]:
# "Doubly Linked" Edge List
class Dles:
    def __init__(self):
        # list of tuples (vertex, id of element of subdivision)
        self.v_list = []
        # face list
        self.f_list = []
        # half_edge list
        self.he_list = []
        
    # self-recongnizing adding vertexes/faces/half_edges
    def add(self, x):
        if type(x).__name__ == 'Vertex':
            self.v_list.append(x)
        elif type(x).__name__ == 'Face':
            self.f_list.append(x)
        elif type(x).__name__ == 'Half_edge':
            self.he_list.append(x)
        

## Converting set of lists of edges into edge list


> This part converts given **lists of edges** only into a set of half_edges list and vertex list, creating the face list later
> 
> Changing to list of points (vertexes) izi if neccessary
>  
> A subdivision is a polygon with possible holes (polygons) inside
>  
> Each vertice has exactly two adjacent edges


In [1]:
# subdivision is list of lists of edges
# subdivision = [polygon1, polygon2, ...] - one is outer boundary, rest are boundaries of holes inside it

def convert( subd ): 

    edge_list = Dles()
    
    # index for half_edges
    edge_index = 0
    e0 = 0
    # neccessary to keep indexes in modulo(length*2) + e0
    def m(x):
         return (x-e0)%(len(polygon)*2) +e0
    # index for vertexes
    vertex_index = 0
    v0 = 0
    def v_i(x):
        return (x-v0)%(len(polygon))+v0
        
    for polygon in subd:
        
        for edge in polygon:
            
            # adding vertex v - beginning of given edge - end is not added
            edge_list.add(Vertex(edge[0], m(edge_index)))
                          
            # adding incident half_edge (starting in v) at index he_i
            edge_list.add(Half_edge(v_i(vertex_index), m(edge_index+1) , m(edge_index+2), m(edge_index-2)))
                          
            #adding the twin of added half_edge with at index he_i+1
            edge_list.add(Half_edge(v_i(vertex_index+1), m(edge_index), m(edge_index-1), m(edge_index+3)))
            
            vertex_index+=1
            edge_index+=2    # i - edge, i+1 - twin edge
            
        e0 = edge_index
        v0 = vertex_index
        
    return edge_list